*Теоретический материал:* https://youtu.be/ffkHDToi-jM?t=237

*Разбор данного раздела:* https://youtu.be/ffkHDToi-jM?t=3407

# **Import библиотек**



In [ ]:
!pip install ffmpeg-python  

In [ ]:
import numpy as np # библиотека для работы с массивами данных
from matplotlib import pyplot as plt # интерфейс для построения графиков простых функций
from IPython.display import Audio # загружаем модуль чтобы обратиться к HTML для записи аудио с микрофона в ноутбуке
from google.colab import files # модуль для загрузки файлов
import librosa # для параметризации аудио
import os # модуль для работы с операционной системой(воспользуемся методами работы с каталогами)
from sklearn.model_selection import train_test_split # модуль для разбивки данных на обучающую и тестовую выборки
from tensorflow.keras.utils import to_categorical # загружаем утилиты кераса для one hot кодировки
from tqdm.notebook import tqdm_notebook as tqdm_
from tensorflow.keras.optimizers import Adam, RMSprop, Adadelta # загружаем алгоритмы обучения/оптимизации
from tensorflow.keras.models import Sequential # последовательная модель нейросети кераса
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Reshape # загружаем необходимые слои для нейросети
import IPython.display as ipd # Воспроизведение звуковыйх файлов
from matplotlib.patches import Rectangle
from IPython.display import HTML, Audio 
import ffmpeg
from google.colab.output import eval_js 
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import scipy


In [ ]:
from google.colab import drive # подключаем гугл драйв диск для загрузки файлов
drive.mount('/content/drive')

# Объявление функций

*Разбор данного раздела:* https://youtu.be/ffkHDToi-jM?t=3836


In [ ]:
# Зададим параметры данных, к-е пойдут на вход нейросети
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/Команды/" # зададим путь к папке с аудиофайлами
DATA_PATH_TEST = "/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/тест/" # Зададим пусть к файлу с тестовыми 30секнудными аудиофайлами

sample_rate = 22050 # Значение sample_rate аудиофайлов
feature_dim_1 = 20 # стандартная величина MFCC признаков
feature_dim_2 = int(.5 * sample_rate) # установленная длина фреймов (в секундах 0.5 = 500 мс)
step_mfcc = int(.02 * sample_rate) # Шаг смещения при разборе mfcc (в секундах 0.1 = 100мс)
channel = 1 # количество каналов
n_classes = 4 # Количество классов команд
classes = ['КОНДИЦИОНЕР', 'СВЕТ', 'ТЕЛЕВИЗОР'] # Классы команд (без фона)


In [ ]:
'''
Зададим функцию извлечения лейблов классов и их обработки
  Параметры:
    path - путь к папке, в которой находятся каталоги с обучающими командами    
'''

def get_labels(path=DATA_PATH): 
  labels = sorted(os.listdir(path)) # запишем лейблы классов по названию папок с данными - ['кондиционер', 'телевизор', 'свет', 'фон']  
  label_indices = np.arange(0, len(labels)) # запишем лейблы в виде индексов - [0, 1, 2, 3]
  return labels, label_indices, to_categorical(label_indices) # функция вернёт лейблы, индексы лейблов и их one-hot представление[0, 1, 0, 0]

In [ ]:
get_labels()[0]

In [ ]:
'''
Создаем функцию параметризации аудио(wav в мел-частоты)
  Параметры:
    length - длинна отрезков, на которые разбиваем исходный файл
    step - шаг, на который смещаем разбиение отрезка
'''

def wav2mfcc(file_path, length = 11025, step = 2205): 
  out_mfcc = [] # Выходной массив, содержащий mfcc исходного файла с шагом step
  out_audio = [] # Выходной массив, содеражищий аудиоинформацию исходного файла с шагом step
  y, sr = librosa.load(file_path) # Загружаем данные исходного файла  

  while (len(y)>=length):  # Проходим весь массив y, пока оставшийся кусочек не станет меньше указанной в параметре max_len длинны
    section = y[:length] # Берем начальный кусок длинной length
    section = np.array(section) # Переводим в numpy
    out_mfcc.append(librosa.feature.mfcc(section, sr)) # Добавляем в выходной массив out_mfcc значение mfcc текущего куска
    out_audio.append(section) # Добавляем в выходной массив аудио текущий кусок
    y = y[step:]  # Уменьшаем y на step
    
  out_mfcc = np.array(out_mfcc) # Преобразуем в numpy
  out_audio = np.array (out_audio) # Преобразуем в numpy
  return out_mfcc, out_audio # функция вернет массив мэл-частот и массив аудио-отрезков

In [ ]:
'''
Объявим функцию формирования и сохранения векторов данных, полученных для каждого набора аудио-команд в датасете
  Параметры:
    path  - путь к папке, в которой находятся каталоги с обучающими командами    
    length - длинна отрезков, на которые разбиваем исходный файл
'''

def save_data_to_array(path=DATA_PATH, length=11025):
  labels, _, _ = get_labels(path) # запишем лейблы классов
  for label in labels: # для каждого лейбла
    mfcc_vectors = [] # здесь соберем векторы MFCC частот
    # извлечем для каждого файла его путь c названием папки и именем файла и соберём в список
    # ['/content/drive/My Drive/Речевые команды/свет/recording12.wav', '/content/drive/My Drive/Речевые команды/свет/recording11.wav'..]
    wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
    print(wavfiles)
    for wavfile in tqdm_(wavfiles, desc=('Сохраняем векторы класса ' + label), ncols=1000):    
      mfcc, _ = wav2mfcc(wavfile, length=length, step = step_mfcc) # получим мел-частоты      
      if (mfcc.shape[0] != 0 ): # Если массив не нулевой длинны
        mfcc_vectors.extend(mfcc) # и добавим вектор в список для соответствующего класса
    np.save('/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/numpy/'+label + '.npy', mfcc_vectors) # сохраним массивы данных для каждого класса

In [ ]:
'''
Создадим функцию для получения обучающей/проверочной выборки
  Параметры (необязательные):
    split_ratio - (1-split_ratio) равно доле тестовых образцов, которые вернет функция train_test_split
    random_state - начальное число, используемое генератором случайных чисел в финкции train_test_split
'''

def get_train_test(split_ratio=0.7, random_state=42, colab = True):
  labels, indices, _ = get_labels(DATA_PATH) # извлекаем имена и индексы лейблов классов

  # Getting first arrays
  X = np.load('/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/numpy/'+labels[0] + '.npy') # берем набор векторов для первого класса 
  y = np.zeros(X.shape[0], dtype = 'int32') # устанавливаем размер соответствующего ему лейбла

  # Объединяем в единый датасет в виде np-массива обучающий и проверочный набор данных
  for i, label in enumerate(labels[1:]): 
    if colab:
      x = np.load(label + '.npy') # Читаем очередной массив данных 
    else:
      x = np.load('/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/numpy/'+label + '.npy') # Читаем очередной массив данных 
    X = np.vstack((X, x)) # Соединяем с исходным набором
    y = np.append(y, np.full(x.shape[0], fill_value= (i + 1))) # В массив y добавлем x.shape[0]-элеменентов со значением (i + 1)

  return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True) # Разделяем данные на обучающую и проверочную выборки

# Подготовка данных

In [ ]:
# Формируем и сохраняем векторы данных, полученных для каждого набора аудио-команд в датасете
save_data_to_array(length=feature_dim_2)

In [ ]:
##########################
# Формируем обучающую и проверочную выборки
##########################
x_train, x_test, y_train, y_test = get_train_test(colab=False) # создаем обучающий/проверочный наборы
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# Добавляем значение канала(=1) в размеры 'x' выборок
#x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], channel)
x_train = x_train[..., None]
#x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], channel)
x_test = x_test[..., None]

y_train_hot = to_categorical(y_train) # представляем лейблы классов обучающего набора в виде one-hot вектора ((0, 1, 0) и т.п)
y_test_hot = to_categorical(y_test) # представляем лейблы классов проверочного набора в виде one-hot вектора ((0, 1, 0) и т.п)

In [ ]:
x_train.shape

In [ ]:
idx = np.where(y_train==0)
idx[0].shape

In [ ]:
idx = np.where(y_train==1)
idx[0].shape

In [ ]:
idx = np.where(y_train==2)
idx[0].shape

In [ ]:
idx = np.where(y_train==3)
idx[0].shape

# Нейросеть

*Разбор данного раздела:* https://youtu.be/ffkHDToi-jM?t=4756

In [ ]:
##########################
# Объявим функцию создания модели нейросети
##########################
def get_model():
  model = Sequential()
  model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], channel)))
  model.add(MaxPooling2D(pool_size=(2, 2))) # передаём на слой подвыборки, снижающий размерность поступивших на него данных 
  model.add(BatchNormalization()) # пропускаем через слой нормализации данных
  model.add(Flatten()) # сплющиваем в одномерный вектор  
  model.add(Dense(128, activation='relu')) # добавляем полносвязный слой размером в заданное кол-во нейронов
  model.add(Dropout(0.25)) # добавляем слой регуляризации, "выключая" указанное количество нейронов, во избежание переобучения
  model.add(BatchNormalization()) # пропускаем через слой нормализации данных
  model.add(Dense(n_classes, activation='softmax')) # добавляем полносвязный слой с функцией активации softmax на выходном слое для 3 классов
  model.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy']) # компилируем, составляем модель с алгоритмом оптимизации, функцией потерь и метрикой точности
  return model

In [ ]:
model = get_model() # создаем модель с помощью объявленной ранее функции

In [ ]:
##########################
# Запустим тренировку нейросети на полученных данных
##########################

# Загружаем модель данными с выбором параметров и сохраним историю обучения(ошибки/точность) для вывода графика
history = model.fit(x_train, y_train_hot, batch_size=256, epochs=30, verbose=1, validation_data=(x_test, y_test_hot))

# Построим график, отображающий точность предсказания на обучающей и проверочной выборке
plt.plot(history.history['accuracy'], label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

# Проверка

### По шагам

*Разбор данного раздела:* https://youtu.be/ffkHDToi-jM?t=4818



In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_cond3.wav' # Получаем имя очередного файла
y,sr = librosa.load(wavfiles) # Загружаем данные модулем librosa
print ('Оригинальный файл:')
ipd.Audio(data=y, rate = sr) # Выводим оригинальный аудиофайл (Команда "Кондиционер" начинается на 12 секунде)

In [ ]:
mfcc_full, audio_full = wav2mfcc (wavfiles, length=feature_dim_2, step = step_mfcc) # Получаем массив mfcc выбранного файла с именем namefile 

mfcc_full = mfcc_full.reshape(-1, 20, 22, 1) # Решейпим для сверточного слоя
g_pred = model.predict(mfcc_full) # Предиктим с помощью модели model массив mfcc
pred = np.array([np.argmax(i) for i in g_pred]) # Выбираем индекс максимального элемента в каждом g_pred[i]  и создаем numpy-массив pred из этих индексов

In [ ]:
print(g_pred.shape)
print(pred.shape)
print(pred[:800])

In [ ]:
idxs = np.where(pred == 0) # В массиве pred находим все элементы со значением, равным искомому классу idx_class
idxs = idxs[0] # Размерность полученного маасива в np.where иммет тип (x,). Оставляем только первую размерность

print (idxs)

In [ ]:
curr = [] # Временный массив для хранения информации о найденных командах
'''
в массиве idx данные могут быть прдеставлены следующим образом:
[4, 5, 6, 7, 123, 124, 125, 126, 127]
в массив curr мы запишем [4, 123] только стартовые индексы
поскольку очевидно, что 4,5,6,7 и 123,124,125,126,127 представляют единую команду
'''
curr_idx =int (idxs[0]) # Текущий стартовый индекс
summ, length = 0, 0 # summ - хранит сумму вероятностей, с которой сеть отнесла команду к данному классу; length - длинна последовательно идущих элементов для одной команды (для массива curr из примера                                                                        
                                                              #[4, 123] длина соответствующая первому элементу будет 4, второму - 5 )
for i in range(len(idxs)): # Пробегаем по всему массиву idxs
  summ += g_pred[idxs[i]][0] # Считаем сумму вероятности
  length += 1 # Увеличиваем длинну последовательности
  if i == len(idxs)-1: # Если последний элемент последовательности
    if (length >= 2 and summ / length >= 0.95): # Проверяем на условия разбора: длинна последовательности должна быть больше входного параметра min_count
                                                        # summ / length должно быть больше входного параметра rate
      curr.append([curr_idx, length, summ / length]) # Если условия выполняются, то добавляем в маасив стартовый индекс найденной команды, длинну последовательности и summ / length
    break        
  if idxs[i+1]-idxs[i]>1: # Если следующий индекс больше текущего на 1 (означает, что следующий элемент относится уже к другой комманде)
    if (length >= 8 and summ / length >= 0.95): # Проверяем на условия разбора: длинна последовательности должна быть больше входного параметра min_count
                                                        # summ / length должно быть больше входного параметра rate
      curr.append([curr_idx, length, summ / length]) # Если условия выполняются, то добавляем в маасив стартовый индекс найденной команды, длинну последовательности и summ / length
    curr_idx = int (idxs[i+1]) # Изменяем текущий стартовый индекс
    summ, length = 0, 0 # Обнуляем summ и length
print (curr)

In [ ]:
np_Audio = []
curr_audio = [] # mfcc отдельной команды
for elem in curr: # Проходим по всему массиву curr
  curr_audio = audio_full[elem[0]]
  if (elem[0] != 0): # Если это не самый первый элемент исходных данных, то возьмем на одну mfcc левее (чаще всего там будет либо тишина, либо начало команды, которое не разобралось сетью)
    curr_audio = np.hstack((audio_full[elem[0] - 1], audio_full[elem[0]][-step_mfcc:]))        
  for j in range(1,elem[1]): # Пробегаем цикл от 1 до elem[1] (где elem[1] хранит длинну последовательности элементов, отнесенных к одной команде)
    if (elem[0]+j == len(audio_full)): # Если elem[0] + j равно длинне mfcc, то выходим из цикла
      break    
    curr_audio = np.hstack((curr_audio, audio_full[elem[0]+j][-step_mfcc:]))
  curr_audio = np.array(curr_audio) # Переводим массив в numpy
  np_Audio.append([curr_audio, elem[2]]) # Добавляем данные в выходной массив

print (len(curr_audio))


In [ ]:
if (len(np_Audio)==0): # Если длина массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
for elem in np_Audio: # Пробегам по всем элементам массива out
    print ('Распознана команда: "', classes[0], '" (вероятность - %.2f' % (elem[1]*100), '%)') # Выводим название
    ipd.display(ipd.Audio (elem[0], rate = 22050))

### Функция

*Разбор данного раздела:* https://youtu.be/ffkHDToi-jM?t=5446


In [ ]:
##########################
# Объявим функцию предсказания команды
##########################
def predict(namefile, model, min_count = 2, rate = 0.9, hole = 1):                    # функция принимает на вход путь к нужному файлу, и имя обученной модели 
    mfcc_full, audio_full = wav2mfcc(namefile, length=feature_dim_2, step = step_mfcc)  # Получаем массив mfcc выбранного файла с именем namefile    

    #mfcc = xScaler.transform(mfcc_full.reshape(-1,1))
    mfcc_full = mfcc_full.reshape(-1, 20, 22, 1)
    g_pred = model.predict(mfcc_full)               # Предиктим с помощью модели model массив mfcc
    pred = np.array([np.argmax(i) for i in g_pred]) # Выбираем индекс максимального элемента в каждом g_pred[i]  и создаем numpy-массив pred из этих индексов

    out = []    # Объявляем выходную переменную out (В ней будут храниться преобразованные из mfcc ауидоданные, класс команды и точность, с которой сеть считает эту команду верной)  
  
    # Ищем команды каждого класса
    for idx_class in range(n_classes-1):
      idxs = np.where(pred == idx_class)  # В массиве pred находим все элементы со значением, равным искомому классу idx_class
      idxs = idxs[0]                      # Размерность полученного маасива в np.where иммет тип (x,). Оставляем только первую размерность
      if (len(idxs) == 0):                # Если элементы искомого класса не найдены,
        continue                          # то переходим к поиску команд следующего класса

      curr = [] # Временный массив для хранения информации о найденных командах
      '''
      в массиве idx данные прдеставлены следующим образом:
      [4, 5, 6, 7, 123, 124, 125, 126, 127]
      в массив curr мы запишем [4, 123] только стартовые индексы
      поскольку очевидно, что 4,5,6,7 и 123,124,125,126,127 представляют единую команду
      '''
      curr_idx =int(idxs[0])  # Текущий стартовый индекс
      summ, length = 0, 0     # summ - хранит сумму вероятностей, с которой сеть отнесла команду к данному классу; length - длинна последовательно идущих элементов для одной команды (для массива curr из примера                                                                        
                              #[4, 123] длина соответствующая первому элементу будет 4, второму - 5 )
      
      for i in range(len(idxs)):              # Пробегаем по всему массиву idxs
        summ += g_pred[idxs[i]][idx_class]    # Считаем сумму вероятности
        length += 1                           # Увеличиваем длинну последовательности
        if i == len(idxs)-1:                  # Если последний элемент последовательности
          if (length >= min_count and summ / length >= rate):   # Проверяем на условия разбора: длинна последовательности должна быть больше входного параметра min_count
                                                                # summ / length должно быть больше входного параметра rate
            curr.append([curr_idx, length, summ / length])      # Если условия выполняются, то добавляем в маасив стартовый индекс найденной команды, длинну последовательности и summ / length

          break  
        if idxs[i+1]-idxs[i]>hole:                            # Если следующий индекс больше текущего на hole (означает, что следующий элемент относится уже к другой комманде)
          if (length >= min_count and summ / length >= rate): # Проверяем на условия разбора: длинна последовательности должна быть больше входного параметра min_count
                                                              # summ / length должно быть больше входного параметра rate
 #           print(length)
            curr.append([curr_idx, length, summ / length])  # Если условия выполняются, то добавляем в маасив стартовый индекс найденной команды, длинну последовательности и summ / length
          curr_idx = int (idxs[i+1])                        # Изменяем текущий стартовый индекс
          summ, length = 0, 0                               # Обнуляем summ и length
      curr_audio = [] 
      for elem in curr:                                     # Проходим по всему массиву curr
        curr_audio = audio_full[elem[0]]      # Если это стартовый элемент исходных данных, то берем самую первую mfcc
        for j in range(1,elem[1]):            # Пробегаем цикл от 1 до elem[1]+1 (где elem[1] хранит длинну последовательности элементов, отнесенных к одной команде)
         if (elem[0]+j == len(audio_full)):   # Если elem[0] + j равно длинне mfcc, то выходим из цикла
           break
         curr_audio = np.hstack((curr_audio, audio_full[elem[0]+j][-step_mfcc:]))
        curr_audio = np.array(curr_audio)             # Переводим массив в numpy
        out.append([curr_audio, idx_class, elem])  # Добавляем данные в выходной массив
    return out, pred                         # Возращаем массив с данными, массив с классами команд, массив с softmax данными

In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_cond14.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 


In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_cond7.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 


In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_televizor'+str(12)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 4, rate = 0.7, hole = 1)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
# Уменьшим параметр rate до 0.75
wavfiles =  DATA_PATH_TEST + 'text_with_svet'+str(1)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

# LIGHT

In [ ]:

# Функция для записи с микрофона из Python
def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  riff_chunk_size = len(output) - 8
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)
  riff = output[:4] + bytes(b) + output[8:]
  sr, audio = wav_read(io.BytesIO(riff))
  return audio, sr

# Скрипт html для записи речи
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

# Запись голоса
# audio, sr = get_audio()
# scipy.io.wavfile.write('recording.wav', sr, audio)

## LIGHT 1

In [ ]:
y_org,sr = get_audio()
scipy.io.wavfile.write('recording.wav', sr, y_org)

In [ ]:
wavfiles = '/content/recording.wav'
y_org,sr = librosa.load(wavfiles) 
ipd.display(ipd.Audio (y_org, rate = sr))
print()
wavfiles = '/content/recording.wav'
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 4, rate = 0.7, hole = 1)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

кондиционер не распознался

In [ ]:
y_org,sr = get_audio()
scipy.io.wavfile.write('recording.wav', sr, y_org)

In [ ]:
wavfiles = '/content/recording.wav'
y_org,sr = librosa.load(wavfiles) 
ipd.display(ipd.Audio (y_org, rate = sr))
print()
wavfiles = '/content/recording.wav'
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 4, rate = 0.7, hole = 1)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

Телевизор не распознался

In [ ]:
y_org,sr = get_audio()
scipy.io.wavfile.write('recording.wav', sr, y_org)

In [ ]:
wavfiles = '/content/recording.wav'
y_org,sr = librosa.load(wavfiles) 
ipd.display(ipd.Audio (y_org, rate = sr))
print()
wavfiles = '/content/recording.wav'
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 4, rate = 0.7, hole = 1)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

свет не распознался

### Вывод

Нейронка очень голосозависима. Из трех примеров распозналс только один наполовину

## LIGHT 2

Запишем файлы сразу из колаба в гугл диск

кондиционер

In [ ]:
for i in range(20):
  y_org,sr = get_audio()
  scipy.io.wavfile.write('/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/Команды/1_cond/my_cond'+str(i)+'.wav', sr, y_org)

свет

In [ ]:
for i in range(20):
  y_org,sr = get_audio()
  scipy.io.wavfile.write('/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/Команды/2_light/my_light'+str(i)+'.wav', sr, y_org)

телевизор

In [ ]:
for i in range(20):
  y_org,sr = get_audio()
  scipy.io.wavfile.write('/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/Команды/3_tv/my_tv'+str(i)+'.wav', sr, y_org)

шум

In [ ]:
for i in range(10,20):
  y_org,sr = get_audio()
  scipy.io.wavfile.write('/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/Команды/4_noise/my_noise'+str(i)+'.wav', sr, y_org)

Переформируем выборки и обучим сеть заново

In [ ]:
# Формируем и сохраняем векторы данных, полученных для каждого набора аудио-команд в датасете
save_data_to_array(length=feature_dim_2)

In [ ]:
##########################
# Формируем обучающую и проверочную выборки
##########################
x_train, x_test, y_train, y_test = get_train_test(colab=False) # создаем обучающий/проверочный наборы
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# Добавляем значение канала(=1) в размеры 'x' выборок
#x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], channel)
x_train = x_train[..., None]
#x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], channel)
x_test = x_test[..., None]

y_train_hot = to_categorical(y_train) # представляем лейблы классов обучающего набора в виде one-hot вектора ((0, 1, 0) и т.п)
y_test_hot = to_categorical(y_test) # представляем лейблы классов проверочного набора в виде one-hot вектора ((0, 1, 0) и т.п)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
idx = np.where(y_train==0)
idx[0].shape

In [ ]:
idx = np.where(y_train==1)
idx[0].shape

In [ ]:
idx = np.where(y_train==2)
idx[0].shape

In [ ]:
idx = np.where(y_train==3)
idx[0].shape

In [ ]:
model = get_model() # создаем модель с помощью объявленной ранее функции

In [ ]:
##########################
# Запустим тренировку нейросети на полученных данных
##########################

# Загружаем модель данными с выбором параметров и сохраним историю обучения(ошибки/точность) для вывода графика
history = model.fit(x_train, y_train_hot, batch_size=256, epochs=50, verbose=1, validation_data=(x_test, y_test_hot))

# Построим график, отображающий точность предсказания на обучающей и проверочной выборке
plt.plot(history.history['accuracy'], label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

проверяем доученную модель

In [ ]:
y_org1,sr1 = get_audio()
scipy.io.wavfile.write('recording1.wav', sr1, y_org1)

In [ ]:
# y_org,sr = get_audio()
# scipy.io.wavfile.write('recording1.wav', sr, y_org)
wavfiles = '/content/recording1.wav'
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 20, rate = 0.8, hole = 4)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
y_org1,sr1 = get_audio()
scipy.io.wavfile.write('recording1.wav', sr1, y_org1)

In [ ]:
# y_org,sr = get_audio()
# scipy.io.wavfile.write('recording1.wav', sr, y_org)
wavfiles = '/content/recording1.wav'
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 20, rate = 0.8, hole = 4)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
y_org1,sr1 = get_audio()
scipy.io.wavfile.write('recording1.wav', sr1, y_org1)

In [ ]:
# y_org,sr = get_audio()
# scipy.io.wavfile.write('recording1.wav', sr, y_org)
wavfiles = '/content/recording1.wav'
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 20, rate = 0.8, hole = 4)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

Выводы: после дообучения с моим голосом сеть стала распознавать мои команды

# PRO

Провел следующие изменения:
- уменьшил проверочную выборку (чтобы чуть увеличить обучающую)
- обрезал часть данных с шумами, для выравнивания базы по объему каждого класса
- чуть усложнил сеть, чтобы выцеплялось больше признаков в mfcc

In [ ]:
##########################
# Формируем обучающую и проверочную выборки
##########################
x_train, x_test, y_train, y_test = get_train_test(split_ratio = 0.85,colab=False) # создаем обучающий/проверочный наборы
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# Добавляем значение канала(=1) в размеры 'x' выборок
#x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], channel)
x_train = x_train[..., None]
#x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], channel)
x_test = x_test[..., None]

y_train_hot = to_categorical(y_train) # представляем лейблы классов обучающего набора в виде one-hot вектора ((0, 1, 0) и т.п)
y_test_hot = to_categorical(y_test) # представляем лейблы классов проверочного набора в виде one-hot вектора ((0, 1, 0) и т.п)

In [ ]:
x_train.shape

In [ ]:
idx = np.where(y_train==0)
idx[0].shape

In [ ]:
idx = np.where(y_train==1)
idx[0].shape

In [ ]:
idx = np.where(y_train==2)
idx[0].shape

In [ ]:
idx = np.where(y_train==3)
idx[0].shape

Обрезаем шумы для выравнивания наборов

In [ ]:
x_train_nois = x_train[y_train==3]
x_train = x_train[y_train!=3]
x_train = np.vstack((x_train, x_train_nois[:3100]))

In [ ]:
y_train_nois = y_train_hot[y_train==3]
y_train_hot = y_train_hot[y_train!=3]
y_train_hot = np.vstack((y_train_hot, y_train_nois[:3100]))

вносим изменения в НС

In [ ]:
##########################
# Объявим функцию создания модели нейросети
##########################
def get_model():
  model = Sequential()
  model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], channel)))
  model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], channel)))
  model.add(MaxPooling2D(pool_size=(2, 2))) # передаём на слой подвыборки, снижающий размерность поступивших на него данных 
  model.add(BatchNormalization()) # пропускаем через слой нормализации данных
  model.add(Flatten()) # сплющиваем в одномерный вектор  
  model.add(Dense(256, activation='relu')) # добавляем полносвязный слой размером в заданное кол-во нейронов
  model.add(Dropout(0.5)) # добавляем слой регуляризации, "выключая" указанное количество нейронов, во избежание переобучения
  model.add(BatchNormalization()) # пропускаем через слой нормализации данных
  model.add(Dense(n_classes, activation='softmax')) # добавляем полносвязный слой с функцией активации softmax на выходном слое для 3 классов
  model.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy']) # компилируем, составляем модель с алгоритмом оптимизации, функцией потерь и метрикой точности
  return model

In [ ]:
model = get_model() # создаем модель с помощью объявленной ранее функции

In [ ]:
##########################
# Запустим тренировку нейросети на полученных данных
##########################

# Загружаем модель данными с выбором параметров и сохраним историю обучения(ошибки/точность) для вывода графика
history = model.fit(x_train, y_train_hot, batch_size=256, epochs=50, verbose=1, validation_data=(x_test, y_test_hot))

# Построим график, отображающий точность предсказания на обучающей и проверочной выборке
plt.plot(history.history['accuracy'], label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

## Проверяем на тестовых файлах с кондиционерами

In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_cond14.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 


In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_cond7.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 


In [ ]:
wavfiles =  '/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/тест/text_with_cond4.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 


In [ ]:
wavfiles =  '/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/тест/text_with_cond16.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 


In [ ]:
wavfiles =  '/content/drive/MyDrive/Colab Notebooks/33 Голосовые команды/тест/text_with_cond10.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 


## Проверяем на тестовых файлах с телевизором

In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_televizor'+str(12)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_televizor'+str(1)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_televizor'+str(2)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_televizor'+str(3)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_televizor'+str(4)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_televizor'+str(5)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
wavfiles =  DATA_PATH_TEST + 'text_with_televizor'+str(6)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

## Проверяем на тестовых данных со светом

In [ ]:
# Уменьшим параметр rate до 0.75
wavfiles =  DATA_PATH_TEST + 'text_with_svet'+str(12)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
# Уменьшим параметр rate до 0.75
wavfiles =  DATA_PATH_TEST + 'text_with_svet'+str(1)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 

In [ ]:
# Уменьшим параметр rate до 0.75
wavfiles =  DATA_PATH_TEST + 'text_with_svet'+str(2)+'.wav' # Получаем имя очередного файла
y_org,sr = librosa.load(wavfiles)      
ipd.display(ipd.Audio (y_org, rate = sr))
print()
#out, pred, _ = predict(wavfiles, model=model, min_count = 6, rate = 0.8, hole = 2)  # Вызываем predict для очередного файла  
out, pred = predict(wavfiles, model=model, min_count = 10, rate = 0.9, hole = 2)                # Вызываем predict для очередного файла  
color = ['y','g','b']
if (len(out)==0):                                                                                   # Если длинна массива равна 0, то команда не распознана
    print('Команда не распознанана!!!')
else:
  for elem in out[:1]: 
    print ('Распознана команда: "', classes[elem[1]], '" (вероятность - %.2f' % (elem[2][2]*100), '%)')  # Выводим название
    ipd.display(ipd.Audio(elem[0], rate = 22050))
    fig = plt.figure(figsize=(30,5))
    ax = fig.add_subplot(111) 
    plt.plot(y_org)
    ax.add_patch(Rectangle((441*elem[2][0], 1.), 
                            20000, -2, 
                              fc =color[elem[1]],  
                              ec =color[elem[1]], 
                              fill=True,
                              lw = 2,
                              alpha=0.5) ) 